In [7]:
from azure.storage.blob import BlobServiceClient
from io import StringIO
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

connection_string = "DefaultEndpointsProtocol=https;AccountName=thesisdata;AccountKey=Br+MEup6o2mHNNh7LNFc564IdrwuZR4b3jKbednry2aDEGRquu2jM1qrcTik86fRaGUQqlJlx4UF+ASt164etg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

container_name = "stock"
blob_name = 'msft_stock_data.csv'

blob_url = f"https://{blob_service_client.primary_hostname}/{container_name}/{blob_name}"

blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
data = blob_client.download_blob().readall().decode("utf-8")

In [8]:
df = pd.read_csv(StringIO(data))
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [9]:
model = SARIMAX(df['Close'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
results = model.fit()


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.99289D+00    |proj g|=  1.86513D-01


 This problem is unconstrained.



At iterate    5    f=  1.87412D+00    |proj g|=  3.15747D-02

At iterate   10    f=  1.85553D+00    |proj g|=  2.64605D-03

At iterate   15    f=  1.85447D+00    |proj g|=  2.63093D-04

At iterate   20    f=  1.85447D+00    |proj g|=  1.06030D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     23     28      1     0     0   1.308D-05   1.854D+00
  F =   1.8544730158980629     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [10]:
forecast = results.get_forecast(steps=30)
predicted_values = forecast.predicted_mean


In [11]:
predicted_dates = pd.date_range(start=df.index[-1], periods=30, freq='D')

predicted_df = pd.DataFrame({'Date': predicted_dates, 'Predicted_Value': predicted_values})

predicted_df.to_csv('predicted_values.csv', index=False)